In [11]:
import numpy as np
import os
from os.path import isfile
from PIL import Image as Img
from datetime import datetime

import keras
from keras.models import Sequential, Model
from keras.utils import plot_model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Flatten, MaxPooling2D, Reshape
from keras.layers import Conv2D, BatchNormalization, Lambda, Permute, GRU
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras import regularizers

import librosa
import librosa.display
import matplotlib.pyplot as plt

import tensorflow as tf

import sys
sys.path.insert(0,'..')

from pianition.data_util import load_dataset
from pianition.keras_models import conv1d_gru

In [12]:
def get_callbacks(checkpoint_name):
    logDir = "./Graph/" + datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
    tb = TensorBoard(log_dir=logDir,
                     histogram_freq=2,
                     write_graph=True,
                     write_images=True,
                     write_grads=True,
                     update_freq='epoch')

    checkpoint_callback = ModelCheckpoint('../models/' + checkpoint_name +
                                          '{epoch:02d}-{val_loss:.2f}.hdf5',
                                          monitor='val_acc',
                                          verbose=1,
                                          save_best_only=True,
                                          mode='max')

    reducelr_callback = ReduceLROnPlateau(monitor='val_acc',
                                          factor=0.5,
                                          patience=10,
                                          min_delta=0.01,
                                          verbose=1)

    callback_list = [checkpoint_callback, reducelr_callback, tb]

    return callback_list

In [21]:
# Data iterator
dataset = load_dataset("../data/debug/")
train_x, train_y = dataset.get_train_full()
val_x, val_y = dataset.get_val_full()

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

print(np.asarray(train_x).shape)

#Create your model
train_model = RNN((128, 256))
#compile
train_model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(lr=0.001),
                    metrics=['accuracy'])
#Train the model
# Better to change checkpoint name before run
train_model.fit(train_x,
                train_y,
                batch_size=32,
                epochs=70,
                validation_data=(val_x, val_y),
                callbacks=get_callbacks(checkpoint_name="trail"))

loaded dataset from ../data/debug/
(494, 128, 256)
(?, 128, 256)
(?, 256, 128)
Train on 494 samples, validate on 150 samples
Epoch 1/70
494/494 [==============================] - 9s 18ms/step - loss: 2.3953 - acc: 0.1437 - val_loss: 2.3648 - val_acc: 0.1467

Epoch 00001: val_acc improved from -inf to 0.14667, saving model to ../models/trail01-2.36.hdf5


KeyboardInterrupt: 

In [ ]:
train_model.summary()